# AI Presentation Assistant — Kaggle Starter Notebook

This notebook is a **Kaggle-ready** starter that demonstrates:
1. Generating a simple PowerPoint (`python-pptx`) from a prompt.
2. A lightweight offline TTS example using `pyttsx3` to synthesize speaker notes to WAV.
3. A simple slideshow-to-video stitch using `moviepy` (skeleton).

Save path: `/kaggle/working/ai_presentation_assistant_demo.*`

Notes:
- On Kaggle you may need to `pip install` the packages in the first code cell.
- If `pyttsx3` does not work on a particular runtime, consider using a small HF TTS model instead.


In [ ]:
# Install minimal dependencies (run on Kaggle environment)
import sys
print('Python', sys.version)
!
pip_args = [
    'python-pptx==0.6.21',
    'pillow',
    'moviepy',
    'pyttsx3',
    'whisper @ git+https://github.com/openai/whisper.git',
    'torch'  # Kaggle already has torch on GPU-enabled runtimes, but include for completeness
]
import subprocess
import sys
for p in pip_args:
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', p], check=True)
    except subprocess.CalledProcessError as e:
        print('Could not install', p, ' — you can skip or install manually on Kaggle runtime')


## 1) Simple PPTX generation demo

This uses `python-pptx` and a tiny helper function to create a multi-slide deck from a prompt.

In [ ]:
from pptx import Presentation
from pptx.util import Inches
import os

def create_basic_presentation(slide_items, out_path='/kaggle/working/demo_output.pptx'):
    prs = Presentation()
    for s in slide_items:
        slide_layout = prs.slide_layouts[1]  # Title and content
        slide = prs.slides.add_slide(slide_layout)
        if slide.shapes.title:
            slide.shapes.title.text = s.get('title', '')
        body = slide.shapes.placeholders[1].text_frame
        body.clear()
        for i, b in enumerate(s.get('bullets', [])):
            if i == 0:
                body.text = b
            else:
                p = body.add_paragraph()
                p.text = b
    prs.save(out_path)
    return out_path

# Demo usage
prompt = 'AI Presentation Assistant — Demo'
slides = []
for i in range(5):
    slides.append({
        'title': f'{prompt} — Slide {i+1}',
        'bullets': [f'Main point {i+1}', 'Supporting detail', 'Example']
    })
out = create_basic_presentation(slides, out_path='/kaggle/working/demo_output.pptx')
print('Saved demo PPT to', out)


## 2) Lightweight offline TTS example (pyttsx3)

`pyttsx3` is an offline TTS that is simple to use. It may not have the highest fidelity, but it's very lightweight and works without external services.

This example will synthesize a short speaker-note string and save it to a WAV file.


In [ ]:
import pyttsx3
import os

def synthesize_to_wav(text, out_wav='/kaggle/working/demo_speaker_notes.wav', rate=150, voice=None):
    """Synthesize text to WAV using pyttsx3. Returns output path.
    Note: On some environments pyttsx3 may not save exactly as WAV; if that occurs you can use SoundFile or pydub to convert formats."""
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)
    if voice:
        engine.setProperty('voice', voice)
    # pyttsx3 supports save_to_file
    engine.save_to_file(text, out_wav)
    engine.runAndWait()
    return out_wav

demo_notes = 'Hello — this is a demo of the AI Presentation Assistant. These are speaker notes for slide one.'
wav_path = synthesize_to_wav(demo_notes, out_wav='/kaggle/working/demo_speaker_notes.wav')
print('Saved TTS WAV to', wav_path)


## 3) Simple slideshow -> video skeleton

This demonstrates how to combine a static avatar image or slide images with the generated audio. For a quick Kaggle demo we create a slideshow where each slide image is shown for an equal portion of the narration audio duration.


In [ ]:
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips
from pathlib import Path

# For demo purposes create 2 slide images (simple colored PNGs) using Pillow
from PIL import Image, ImageDraw, ImageFont
os.makedirs('/kaggle/working/slide_images', exist_ok=True)
for i, color in enumerate([(255,255,255),(230,245,255)], start=1):
    img = Image.new('RGB', (1280,720), color=color)
    d = ImageDraw.Draw(img)
    d.text((60,60), f'Demo Slide {i}', fill=(0,0,0))
    p = f'/kaggle/working/slide_images/slide_{i}.png'
    img.save(p)

audio_path = '/kaggle/working/demo_speaker_notes.wav'
if not Path(audio_path).exists():
    print('Audio not found at', audio_path, ' — please run the TTS cell or upload an audio file named demo_speaker_notes.wav')
else:
    audio_clip = AudioFileClip(audio_path)
    imgs = sorted(Path('/kaggle/working/slide_images').glob('*.png'))
    clips = []
    per_slide = audio_clip.duration / max(1, len(imgs))
    for img_path in imgs:
        ic = ImageClip(str(img_path)).set_duration(per_slide)
        clips.append(ic)
    final = concatenate_videoclips(clips).set_audio(audio_clip)
    out_video = '/kaggle/working/demo_presentation_video.mp4'
    final.write_videofile(out_video, fps=24)
    print('Saved demo video to', out_video)


----
### What I saved for you
- `/kaggle/working/demo_output.pptx` — generated PPT (if you run the PPT cell)
- `/kaggle/working/demo_speaker_notes.wav` — TTS audio (if `pyttsx3` saves correctly in the runtime)
- `/kaggle/working/demo_presentation_video.mp4` — resulting slideshow video after running the video cell

If you want, I can now:
- attach this notebook file for download, or
- additionally add a cell that demonstrates Whisper ASR + evaluation metrics (simple scoring), or
- replace `pyttsx3` with a small HF TTS wrapper (if you prefer model-based TTS).
